Here I am using the request module to access the wikipedia page, converting it to text (printed for reference),
then isolated the table part of the page in variable 'postal'.

In [1]:
import requests
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_tor_wikipedia_page = requests.get(wikipedia_link)
page = raw_tor_wikipedia_page.text
print(page)

postal = page[(page.find('<tbody>')+len('<tbody')):page.find('</tbody>')]

<!DOCTYPE html>
<html class="client-nojs" lang="en" dir="ltr">
<head>
<meta charset="UTF-8"/>
<title>List of postal codes of Canada: M - Wikipedia</title>
<script>document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );</script>
<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":876823784,"wgRevisionId":876823784,"wgArticleId":539066,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","w

I personally use regex and string replace, so I used 5 iterations of that logic to make a pipe delimited
string with line breaks every 3 columns,  Preinted for reference.

In [2]:
import re
p0 =re.sub('<.*?>','|',postal)
p1 =str.replace(p0,'||','|')
p2 =str.replace(p1,'\n|\n|\n|','\n')
p3 =str.replace(p2,'|\n|','|')
p4 =str.replace(p3,'|\n','\n').strip('>|')
p4

"Postcode|Borough|Neighbourhood\nM1A|Not assigned|Not assigned\nM2A|Not assigned|Not assigned\nM3A|North York|Parkwoods\nM4A|North York|Victoria Village\nM5A|Downtown Toronto|Harbourfront\nM5A|Downtown Toronto|Regent Park\nM6A|North York|Lawrence Heights\nM6A|North York|Lawrence Manor\nM7A|Queen's Park|Not assigned\nM8A|Not assigned|Not assigned\nM9A|Etobicoke|Islington Avenue\nM1B|Scarborough|Rouge\nM1B|Scarborough|Malvern\nM2B|Not assigned|Not assigned\nM3B|North York|Don Mills North\nM4B|East York|Woodbine Gardens\nM4B|East York|Parkview Hill\nM5B|Downtown Toronto|Ryerson\nM5B|Downtown Toronto|Garden District\nM6B|North York|Glencairn\nM7B|Not assigned|Not assigned\nM8B|Not assigned|Not assigned\nM9B|Etobicoke|Cloverdale\nM9B|Etobicoke|Islington\nM9B|Etobicoke|Martin Grove\nM9B|Etobicoke|Princess Gardens\nM9B|Etobicoke|West Deane Park\nM1C|Scarborough|Highland Creek\nM1C|Scarborough|Rouge Hill\nM1C|Scarborough|Port Union\nM2C|Not assigned|Not assigned\nM3C|North York|Flemingdon Park

StringIO module is used to write the string as the file.
Pandas read csv recognizes the new lines and assigned pipe-delimited when calle out.
I went ahead and removed the 'Not Assigned' Boroughs as well.

In [3]:
import pandas as pd
from io import StringIO

data = StringIO(p4)
df = pd.read_csv(data, sep='|')
df = df[df.Borough != 'Not assigned']
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


Noted steps I took below in comments

In [4]:
#grouped by the post code and borough, then concatenated neighbourhood with an apply function and reset index
df = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda x: '%s' % ', '.join(x))
df = pd.DataFrame(df).reset_index()

#assigned the borough name where the neighbourhood was missing.
df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df['Borough']
df #here is the DataFrame.  This completes requirements from what I can tell; shape is in cell below.

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [5]:
df.shape

(103, 3)